In [9]:
#Imports
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.feature_extraction.text import TfidfVectorizer

In [10]:
#Load datasets
# movie dataset
mov_data = pd.read_csv('data/movie.csv', delimiter=',', quotechar='"', encoding='utf-8', on_bad_lines='skip')
mov_data = mov_data.to_numpy()
mov_data.shape
mov_X = mov_data[1:, 0] # all rows except the first one, text column
mov_y = mov_data[1:, 1] # all rows except the first one, label column

# chat gpt dataset
cgpt_data = pd.read_csv('data/chatgpt_sentiment_analysis.csv', delimiter=',', quotechar='"', encoding='utf-8', on_bad_lines='skip')
cgpt_data = cgpt_data.to_numpy()
cgpt_data.shape
cgpt_X = cgpt_data[1:, 1] # all rows except the first one, text column
cgpt_y = cgpt_data[1:, 2] # all rows except the first one, label column

# social media dataset
sm_data = pd.read_csv('data/soc_med_sentiment_analysis.csv', delimiter=',', quotechar='"', encoding='utf-8', on_bad_lines='skip')
sm_data = sm_data.to_numpy()
sm_data.shape
sm_X = sm_data[1:, 4] # all rows except the first one, text column
sm_y = sm_data[1:, 5] # all rows except the first one, label column

In [11]:
#Combine X and Y dataframes, then vectorize
X = np.concatenate([mov_X, cgpt_X, sm_X])
y = np.concatenate([mov_y, cgpt_y, sm_y])

vectorizer = TfidfVectorizer(max_features=1000)
X = vectorizer.fit_transform(X).toarray()

In [12]:
#Convert labels in y
y = np.where(y == "good", 1, np.where(y == "neutral", 0, np.where(y == "bad", -1, y)))
y = np.where(y == "positive", 1, np.where(y == "neutral", 0, np.where(y == "negative", -1, y)))

In [13]:
#Preprocessing
def preprocess(X, y):
    # Normalize the data
    scaler = StandardScaler()
    X = scaler.fit_transform(X)

    # Shuffle the data
    indices = []

    for i in range(len(X)):
        indices.append(i)

    np.random.shuffle(indices)
    X = X[indices]
    y = y[indices]

    #Half the amount of data
    split = int(0.5 * X.shape[0])
    X = X[:split]
    y = y[:split]

    # Split data: training 80%, testing 20%
    split = int(0.8 * X.shape[0])
    X_train, X_test = X[:split], X[split:]
    y_train, y_test = y[:split], y[split:]

    # return X, X_train, X_test, y, y_train, y_test
    return X_train, X_test, y_train, y_test

X_train, X_test, y_train, y_test = preprocess(X, y)

In [ ]:
#Model
class BackPropagation:
    def __init__(self, input_size, hidden_size, output_size):
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.output_size = output_size
        
        # Initialize weights and biases
        self.b1 = np.zeros((1, self.hidden_size))
        self.b2 = np.zeros((1, self.output_size))
        self.W1 = np.random.randn(self.input_size, self.hidden_size) * np.sqrt(2 / self.input_size)
        self.W2 = np.random.randn(self.hidden_size, self.output_size) * np.sqrt(2 / self.hidden_size)


    def sigmoid(self, x):
        return 1 / (1 + np.exp(-x))

    def sigmoid_derivative(self, x):
        return x * (1 - x)

    def relu(self, x):
        return x * (x >= 0)

    def relu_derivative(self, x):
        return 1. * (x > 0)

    def forward(self, X):
        # Hidden layer
        hidden_activation = self.relu(np.dot(X, self.W1) + self.b1)
        # Output layer
        output = np.dot(hidden_activation, self.W2) + self.b2
        return output

    def backward(self, X, y, output, rate):
        # Calculate error (output)
        output_error = (y - output)

        # Calculate hidden activations
        hidden = self.relu(np.dot(X, self.W1) + self.b1)

        # Calculate error (hidden)
        hidden_error = np.dot(output_error, self.W2.T)
        hidden_delta = hidden_error * self.sigmoid_derivative(hidden)

        # Update weights
        self.W2 = self.W2 + rate*(np.dot(hidden.T, output_error))
        self.b2 = self.b2 + rate*(np.sum(output_error, axis=0, keepdims=True))

        self.W1 = self.W1 + rate*(np.dot(X.T, hidden_delta))
        self.b1 = self.b1 + rate*(np.sum(hidden_delta, axis=0, keepdims=True))
        
    def train(self, X, y, learning_rate, epochs):
        y = y.reshape(-1,1)

        for _ in range(epochs):
            print(f"W1 max: {np.max(self.W1)}, min: {np.min(self.W1)}")
            print(f"W2 max: {np.max(self.W2)}, min: {np.min(self.W2)}")
            print(f"B1 max: {np.max(self.b1)}, min: {np.min(self.b1)}")
            print(f"B2 max: {np.max(self.b2)}, min: {np.min(self.b2)}")
            output = self.forward(X)
            self.backward(X,y, output, learning_rate)

    def predict(self, X):
        return self.forward(X)

In [ ]:
#backprop training
input_size = X_train.shape[1]
hidden_size = 10
output_size = 1

bp = BackPropagation(input_size, hidden_size, output_size)
bp.train(X_train, y_train, learning_rate=0.0001, epochs=4)

W1 max: 0.18270504413674243, min: -0.15149077570746194
W2 max: 0.5496137087376242, min: -0.7362625345992023
B1 max: 0.0, min: 0.0
B2 max: 0.0, min: 0.0


TypeError: ufunc 'isinf' not supported for the input types, and the inputs could not be safely coerced to any supported types according to the casting rule ''safe''

In [ ]:
#Predictions
train_predictions = bp.predict(X_train)
test_predictions = bp.predict(X_test)

In [ ]:
train_predictions = train_predictions.flatten()
test_predictions = test_predictions.flatten()

In [ ]:
train_mse = np.mean((train_predictions - y_train) ** 2)
test_mse = np.mean((test_predictions - y_test) ** 2)

print(f"Train MSE: {train_mse}")
print(f"Test MSE: {test_mse}")

Train MSE: inf
Test MSE: inf
